<a href="https://colab.research.google.com/github/wonyoung94/-/blob/master/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
from google.colab import files
files.upload()
# 파일 선택 창이 뜨면 test, train 파일을 넣어주세요.

Saving test.csv to test.csv
Saving train.csv to train.csv


{'test.csv': b'user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern\n0001d6e9,9,15,11.173050771674038,7.047243131461066,19,15,82.07455731157792,3,5,Low,Premium,2,7\n0002c77d,11,8,13.448592422239834,24.583327332881424,10,16,80.54604030048435,4,2,Low,Basic,4,4\n0002df5b,11,28,18.548201320577764,1.707615797262084,9,19,73.67695729808806,5,5,Low,Premium,1,3\n000b6068,14,17,13.383310984712208,21.391389159924124,12,6,63.31463468304937,0,4,High,Premium,1,5\n00184a0c,19,6,11.919663015192976,18.126987229641152,8,15,55.899646880696906,1,4,Medium,Basic,1,3\n001d2775,13,23,14.16548322643258,47.94771321025265,14,13,73.57392418742418,3,5,Medium,Premium,3,1\n00227b7e,12,16,7.925745431050987,19.67798367381445,13,10,87.4311036014169,3,1,M

# 컬럼 설명 column	description
- user_id	사용자의 고유 식별자
- subscription_duration	사용자가 서비스에 가입한 기간 (월)
- recent_login_time	사용자가 마지막으로 로그인한 시간 (일)
- average_login_time	사용자의 일반적인 로그인 시간 : **홈페이지에 로그인 상태를 유지한 시간**
- average_time_per_learning_session	각 학습 세션에 소요된 평균 시간 (분)
- monthly_active_learning_days	월간 활동적인 학습 일수
- total_completed_courses	완료한 총 코스 수
- recent_learning_achievement	최근 학습 성취도
- abandoned_learning_sessions	중단된 학습 세션 수
- community_engagement_level	커뮤니티 참여도
- preferred_difficulty_level	선호하는 난이도
- subscription_type	구독 유형
- customer_inquiry_history	고객 문의 이력
- payment_pattern	사용자의 지난 3개월 간의 결제 패턴을 10진수로 표현한 값.
 - 7: 3개월 모두 결제함
 - 6: 첫 2개월은 결제했으나 마지막 달에는 결제하지 않음
 - 5: 첫 달과 마지막 달에 결제
 - 4: 첫 달에만 결제
 - 3: 마지막 2개월에 결제함
 - 2: 가운데 달에만 결제함
 - 1: 마지막 달에만 결제함
 - 0: 3개월 동안 결제하지 않음
target	사용자가 다음 달에도 구독을 계속할지 (1) 또는 취소할지 (0)를 나타냄

In [3]:
# 데이터프레임 선언
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
del train_data['target']

In [4]:
# data type 확인 및 결측치 확인
train_data.info()
test_data.info()
train_data.isnull().sum() #결측치 없음
test_data.isnull().sum() #결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   user_id                            10000 non-null  object 
 1   subscription_duration              10000 non-null  int64  
 2   recent_login_time                  10000 non-null  int64  
 3   average_login_time                 10000 non-null  float64
 4   average_time_per_learning_session  10000 non-null  float64
 5   monthly_active_learning_days       10000 non-null  int64  
 6   total_completed_courses            10000 non-null  int64  
 7   recent_learning_achievement        10000 non-null  float64
 8   abandoned_learning_sessions        10000 non-null  int64  
 9   community_engagement_level         10000 non-null  int64  
 10  preferred_difficulty_level         10000 non-null  object 
 11  subscription_type                  10000 non-null  obje

user_id                              0
subscription_duration                0
recent_login_time                    0
average_login_time                   0
average_time_per_learning_session    0
monthly_active_learning_days         0
total_completed_courses              0
recent_learning_achievement          0
abandoned_learning_sessions          0
community_engagement_level           0
preferred_difficulty_level           0
subscription_type                    0
customer_inquiry_history             0
payment_pattern                      0
dtype: int64

In [5]:
# 수치형 및 범주형 변수 구분
num_test_cols = [col for col in test_data.columns if train_data[col].dtype in ['int64', 'float64']]
num_train_cols = [col for col in train_data.columns if train_data[col].dtype in ['int64', 'float64']]

cat_test_cols = [col for col in test_data.columns if train_data[col].dtype == 'object' and col not in ['user_id']]
cat_train_cols = [col for col in train_data.columns if train_data[col].dtype == 'object' and col not in ['user_id']]

In [6]:
#수치형 데이터
num_test_cols
num_train_cols

['subscription_duration',
 'recent_login_time',
 'average_login_time',
 'average_time_per_learning_session',
 'monthly_active_learning_days',
 'total_completed_courses',
 'recent_learning_achievement',
 'abandoned_learning_sessions',
 'community_engagement_level',
 'customer_inquiry_history',
 'payment_pattern']

In [7]:
#범주형 데이터
cat_test_cols
cat_train_cols

['preferred_difficulty_level', 'subscription_type']

In [10]:
# ★ 모델링 특성에 맞게 실행하지 않으셔도 됩니다. ★
# 데이터 스케일링 : 정규분포화 기준 (분류에 적합한 특성을 고려하여 정규분포화 했습니다.)

# StandardScaler 인스턴스화
scaler = StandardScaler()

# 수치형 데이터에 대해 스케일링 적용
test_data[num_test_cols] = scaler.fit_transform(test_data[num_train_cols])
train_data[num_train_cols] = scaler.fit_transform(train_data[num_train_cols])

# 레이블 인코딩
label_encoders = {}

for col in num_test_cols:
  label_encoders[col] = LabelEncoder()
  test_data[col] = label_encoders[col].fit_transform(test_data[col])

for col in num_train_cols:
  label_encoders[col] = LabelEncoder()
  train_data[col] = label_encoders[col].fit_transform(train_data[col])

In [11]:
# 전처리 후 학습 데이터 확인
test_data.head()

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern
0,0001d6e9,8,14,997,1217,18,14,7566,3,4,Low,Premium,2,7
1,0002c77d,10,7,2971,3651,9,15,7100,4,1,Low,Basic,4,4
2,0002df5b,10,27,8841,275,8,18,4500,5,4,Low,Premium,1,3
3,000b6068,13,16,2894,3251,11,5,1225,0,3,High,Premium,1,5
4,00184a0c,18,5,1474,2821,7,14,275,1,3,Medium,Basic,1,3


In [12]:
train_data.head()

,user_id,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern
0,b919c29d,12,13,4955,1459,17,15,2531,3,3,Low,Basic,4,5
1,a0a60abb,15,17,8746,7362,15,12,9876,2,2,Medium,Basic,1,6
2,b9f171ae,21,0,6572,3363,12,13,9725,3,3,Medium,Premium,0,7
3,5dc0ba8b,0,18,8102,5402,18,17,3128,0,2,Low,Basic,1,0
4,65c83654,3,4,9835,4316,18,9,7561,2,3,Medium,Basic,3,0
